In [2]:
#pip install librosa 

In [3]:
import librosa

In [72]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


# Python script for confusion matrix creation. 
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report

In [5]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
            result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [6]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [7]:
emotions['04']

'sad'

In [8]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size):
    x,y=[],[]

    for file in glob.glob("C:\\Users\\sachin.bhoi\\Documents\\Jupyter_Notebook\\Speech_emotion_recognition\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

#"D:\\DataFlair\\ravdess data\\Actor_*\\*.wav"
#C:\Users\sachin.bhoi\Documents\Jupyter_Notebook\Speech_emotion_recognition\speech-emotion-recognition-ravdess-data


In [9]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [10]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [11]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [12]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [13]:
#DataFlair - Train the model
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [14]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)
y_pred

array(['happy', 'calm', 'disgust', 'calm', 'disgust', 'calm', 'calm',
       'happy', 'calm', 'fearful', 'happy', 'fearful', 'fearful', 'happy',
       'disgust', 'happy', 'calm', 'disgust', 'disgust', 'calm',
       'disgust', 'disgust', 'disgust', 'calm', 'happy', 'happy',
       'fearful', 'happy', 'fearful', 'fearful', 'happy', 'disgust',
       'happy', 'fearful', 'happy', 'calm', 'calm', 'fearful', 'calm',
       'disgust', 'happy', 'disgust', 'fearful', 'calm', 'fearful',
       'disgust', 'disgust', 'calm', 'disgust', 'happy', 'fearful',
       'fearful', 'disgust', 'fearful', 'disgust', 'happy', 'disgust',
       'disgust', 'calm', 'calm', 'disgust', 'calm', 'disgust', 'happy',
       'disgust', 'calm', 'calm', 'disgust', 'disgust', 'happy',
       'fearful', 'fearful', 'fearful', 'fearful', 'fearful', 'disgust',
       'fearful', 'happy', 'calm', 'fearful', 'disgust', 'calm',
       'fearful', 'calm', 'disgust', 'fearful', 'disgust', 'fearful',
       'happy', 'disgust', 'dis

In [75]:
CM = confusion_matrix(y_test, y_pred)
print(CM)

[[32 14  8  3]
 [ 3 32  8  5]
 [ 1  5 27  4]
 [ 7  6 16 21]]


In [15]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 58.33%


In [65]:
Sachin = []     # creates an empty list in python
x = []
for file in glob.glob("C:\\Users\\sachin.bhoi\\Documents\\Jupyter_Notebook\\Speech_emotion_recognition\\Sachin\\*.wav"):   # for each file in this folder
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)    # extract the features of the sound from each file 
        x.append(emotions[file_name.split("-")[2]])          # appends the real emotions, decoded from the file name by the emotions dictionary
        Sachin.append(feature)     # appends the features into the list
        file_name=os.path.basename(file)    # os.path.basename(file), this thing only gives the file name in .wav, ----> file gives all the path of the file location from the root drive            
        emotion=emotions[file_name.split("-")[2]]     # file_name.split("-")[2] this thing gives the value of 3rd (first location starts from 0) digit after "-" in file and the emotions dictionary decodes this code number according to the corrosponding emotion like happy, disguist and save it in emotion  
        print(os.path.basename(file))     #prints the emotion text every time in loop
#Sachin[0]

03-01-03-02-02-02-03.wav
03-01-04-01-01-01-03.wav
03-01-04-01-01-02-03.wav
03-01-04-01-02-01-03.wav


In [71]:
pred = model.predict(Sachin)     # predicts the lable (emotion) from the set of features in the Sachin list
for i in range(len(Sachin)):     # For the number of files, repeat the loop so many times
    print("Real emotion --->", x[i], " Vs the Machine Learning Prediction --->", pred[i]) # Real emotion vs the prediction


Real emotion ---> sad  Vs the Machine Learning Prediction ---> happy
Real emotion ---> happy  Vs the Machine Learning Prediction ---> calm
Real emotion ---> sad  Vs the Machine Learning Prediction ---> calm
Real emotion ---> sad  Vs the Machine Learning Prediction ---> fearful
